# Drone Detection System - Complete Training Pipeline
This notebook will:
1. Mount Google Drive for saving progress
2. Download datasets from Roboflow
3. Train YOLOv8 model
4. Save checkpoints automatically
5. Test the model

## Step 1: Mount Google Drive (IMPORTANT - Run this first!)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project folder in Drive
import os
project_path = '/content/drive/MyDrive/DroneDetection'
os.makedirs(project_path, exist_ok=True)
print(f"✅ Google Drive mounted! Your models will be saved to: {project_path}")

## Step 2: Install Required Libraries

In [ ]:
!pip install ultralytics roboflow -q
print("✅ Libraries installed!")

## Step 3: Download Datasets from Roboflow
You'll need to get API keys for each dataset:
1. Go to each Roboflow link
2. Click "Download" button
3. Select "YOLOv8" format
4. Copy the code snippet they provide (it has your API key)

For now, I'll show you the structure - you'll paste your actual API keys

In [ ]:
from roboflow import Roboflow

# Dataset 1: 7000 images
# Go to: https://universe.roboflow.com/drone-rwsrk/drone-cmxwz
# Click Download > YOLOv8 > Show download code
# Replace 'YOUR_API_KEY_1' with the actual key

rf1 = Roboflow(api_key="YOUR_API_KEY_1")
project1 = rf1.workspace("drone-rwsrk").project("drone-cmxwz")
dataset1 = project1.version(1).download("yolov8", location="/content/dataset1")
print("✅ Dataset 1 downloaded (7000 images)")

# Dataset 2: 1700 images
rf2 = Roboflow(api_key="YOUR_API_KEY_2")
project2 = rf2.workspace("drone-l3ty9").project("drone-6cbn9")
dataset2 = project2.version(1).download("yolov8", location="/content/dataset2")
print("✅ Dataset 2 downloaded (1700 images)")

# Dataset 3: 2500 images
rf3 = Roboflow(api_key="YOUR_API_KEY_3")
project3 = rf3.workspace("mailik-j9cdn").project("dronealone")
dataset3 = project3.version(1).download("yolov8", location="/content/dataset3")
print("✅ Dataset 3 downloaded (2500 images)")

## Step 4: Combine Datasets (Optional but Recommended)
More data = better model! Let's merge all three datasets

In [ ]:
import shutil
import yaml
from pathlib import Path

# Create combined dataset folder
combined_path = Path('/content/combined_dataset')
combined_path.mkdir(exist_ok=True)

for split in ['train', 'valid', 'test']:
    (combined_path / split / 'images').mkdir(parents=True, exist_ok=True)
    (combined_path / split / 'labels').mkdir(parents=True, exist_ok=True)

# Copy all datasets into combined folder
datasets = ['/content/dataset1', '/content/dataset2', '/content/dataset3']
image_counter = 0

for dataset_path in datasets:
    for split in ['train', 'valid', 'test']:
        # Copy images
        img_src = Path(dataset_path) / split / 'images'
        if img_src.exists():
            for img_file in img_src.glob('*'):
                new_name = f"drone_{image_counter}_{img_file.name}"
                shutil.copy(img_file, combined_path / split / 'images' / new_name)
                
                # Copy corresponding label
                label_file = Path(dataset_path) / split / 'labels' / f"{img_file.stem}.txt"
                if label_file.exists():
                    shutil.copy(label_file, combined_path / split / 'labels' / f"drone_{image_counter}_{img_file.stem}.txt")
                
                image_counter += 1

print(f"✅ Combined dataset created with {image_counter} total images!")

# Create data.yaml file for YOLO
data_yaml = {
    'path': str(combined_path),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 1,  # number of classes (just 'drone')
    'names': ['drone']
}

with open(combined_path / 'data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print(f"✅ data.yaml created at {combined_path / 'data.yaml'}")

## Step 5: Train YOLOv8 Model
This will train the model and automatically save checkpoints to Google Drive

In [ ]:
from ultralytics import YOLO
import shutil

# Load a pretrained YOLOv8 model (nano version for faster training)
# Options: yolov8n.pt (nano, fastest), yolov8s.pt (small), yolov8m.pt (medium), yolov8l.pt (large)
model = YOLO('yolov8n.pt')  # Start with nano for quick experiments

# Train the model
results = model.train(
    data=str(combined_path / 'data.yaml'),  # path to data.yaml
    epochs=50,  # number of training epochs (start with 50, increase to 100-200 for better results)
    imgsz=640,  # image size
    batch=16,  # batch size (adjust based on GPU memory - 16 is safe for Colab)
    name='drone_detector',  # name of the run
    patience=10,  # early stopping patience
    save=True,  # save checkpoints
    device=0,  # use GPU
    project=project_path,  # save to Google Drive
)

print("✅ Training complete!")
print(f"✅ Model saved to: {project_path}/drone_detector")

## Step 6: Backup Best Model to Google Drive Root
Extra safety - copy the best model to an easy-to-find location

In [ ]:
import shutil
from datetime import datetime

# Copy best model to Drive with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
best_model_path = f"{project_path}/drone_detector/weights/best.pt"
backup_path = f"{project_path}/drone_best_{timestamp}.pt"

shutil.copy(best_model_path, backup_path)
print(f"✅ Best model backed up to: {backup_path}")
print("\n📊 Training Results Summary:")
print(f"Results folder: {project_path}/drone_detector")
print(f"Best weights: {best_model_path}")
print(f"Last weights: {project_path}/drone_detector/weights/last.pt")

## Step 7: Evaluate the Model

In [ ]:
# Load the best model
best_model = YOLO(f"{project_path}/drone_detector/weights/best.pt")

# Validate on test set
metrics = best_model.val()

print("\n📊 Model Performance Metrics:")
print(f"mAP50: {metrics.box.map50:.3f}")  # Mean Average Precision at IoU=0.5
print(f"mAP50-95: {metrics.box.map:.3f}")  # Mean Average Precision at IoU=0.5:0.95
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

## Step 8: Test on Sample Images
Let's see the model in action!

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import glob

# Get some test images
test_images = glob.glob(str(combined_path / 'test' / 'images' / '*'))[:5]  # First 5 test images

# Run inference
results = best_model.predict(test_images, conf=0.25)  # confidence threshold

# Display results
fig, axes = plt.subplots(1, len(results), figsize=(20, 4))
if len(results) == 1:
    axes = [axes]

for idx, (result, ax) in enumerate(zip(results, axes)):
    img = result.plot()  # plot predictions on image
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"Test Image {idx+1}")

plt.tight_layout()
plt.savefig(f"{project_path}/sample_predictions.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Sample predictions saved to: {project_path}/sample_predictions.png")

## Step 9: Export Model for Deployment (Optional)
Export to different formats for real-time inference

In [ ]:
# Export to ONNX (for faster inference)
onnx_path = best_model.export(format='onnx')
print(f"✅ ONNX model exported: {onnx_path}")

# You can also export to:
# - TensorRT (for NVIDIA GPUs): format='engine'
# - TensorFlow: format='saved_model'
# - TFLite (for mobile): format='tflite'
# - CoreML (for iOS): format='coreml'

## Step 10: Quick Inference Code for Real-Time Detection
Use this code in your final application

In [ ]:
# Example: Real-time detection on webcam or video
# (This won't work in Colab but you can use it in your local app)

inference_code = '''
from ultralytics import YOLO
import cv2

# Load model
model = YOLO('path/to/best.pt')

# For webcam
cap = cv2.VideoCapture(0)

# For video file
# cap = cv2.VideoCapture('drone_video.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run inference
    results = model(frame, conf=0.5)
    
    # Draw results
    annotated_frame = results[0].plot()
    
    # Display
    cv2.imshow('Drone Detection', annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
'''

# Save inference code to Drive
with open(f"{project_path}/inference_example.py", 'w') as f:
    f.write(inference_code)

print(f"✅ Inference example code saved to: {project_path}/inference_example.py")

## 🎉 Training Complete!

### What you have now:
1. ✅ Trained YOLOv8 drone detection model
2. ✅ Best weights saved to Google Drive (safe from session timeouts)
3. ✅ Training metrics and visualizations
4. ✅ Sample predictions
5. ✅ Inference code ready to use

### Next Steps:
1. **Download your model** from Google Drive: `{project_path}/drone_detector/weights/best.pt`
2. **Test with your own videos/images**
3. **Fine-tune**: If results aren't great, try:
   - Increase epochs (100-200)
   - Try larger model (yolov8s.pt or yolov8m.pt)
   - Adjust confidence threshold
4. **Deploy**: Use the inference code for real-time detection

### Resume Training Later:
If your session times out, just run:
```python
model = YOLO(f"{project_path}/drone_detector/weights/last.pt")
model.train(resume=True)
```

### Performance Tips:
- **mAP50 > 0.80**: Excellent
- **mAP50 0.60-0.80**: Good
- **mAP50 < 0.60**: Needs improvement (more epochs/better data)

Good luck with your project! 🚁